In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

Display function

In [ ]:
def display_images(ds):
    class_names = ['Potato___Early_blight', 'Potato___Late_blight', 'Potato___healthy']

    plt.figure(figsize=(10, 10))
    for image_batch in ds.take(1): ## One batch
        for i in range(12):
            ax = plt.subplot(3, 4, i + 1)
            plt.imshow(image_batch[0][i].numpy().astype('uint8'))
            plt.title(class_names[image_batch[1][i]])
            plt.axis('off')

Read Data from director

In [ ]:
height = 256
weight = 256

batch_size = 32
epochs = 50

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    directory='PlantVillage',
    shuffle=True,
    image_size=(height, weight),
    batch_size=batch_size
)

Train, test split with custom function

In [58]:
def get_data_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=1000):
    ds_size = len(ds)

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=55)
    
    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds

train_ds, val_ds, test_ds = get_data_partitions_tf(dataset)

In [ ]:
display_images(train_ds)

In [ ]:
display_images(val_ds)

In [ ]:
display_images(test_ds)

Cache and Prefetch

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

Preprocessing

In [ ]:
resize_rescale = tf.keras.Sequential([
    layers.experimental.preprocessing.Resize(height, weight),
    layers.experimental.preprocessing.Rescaling(1.0 / 255)
])